In [1]:
import time
import os
import pandas as pd
import numpy as np
import json
import matplotlib.pyplot as plt
import seaborn as sns
%config InlineBackend.figure_format ='retina'
import random
from functools import reduce
import spotipy
import spotipy.util as util
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy import oauth2
from tqdm import tqdm

In [2]:
def login():
    cid = '98ecc26f9931438a852afca48a8a0196'
    secret = 'db6462ad1fd4460cb70867c38347a045'
    redirect_uri='http://localhost:7777/callback'
    username = 'washingtondctrip'

    # Once the Authorisation is complete, we just need to `sp` to call the APIs
    scope = 'user-top-read playlist-modify-private playlist-modify-public'
    token = util.prompt_for_user_token(username, scope, client_id=cid, client_secret=secret, redirect_uri=redirect_uri)

    if token:
        sp = spotipy.Spotify(auth=token)
    else:
        print("Can't get token for", username)
    return sp

In [3]:
def getTrackFeatures(track_id):
    sp = login()
    features = sp.audio_features(track_id)
    
    if features and features[0]:
        acousticness = features[0]['acousticness']
        danceability = features[0]['danceability']
        energy = features[0]['energy']
        instrumentalness = features[0]['instrumentalness']
        liveness = features[0]['liveness']
        loudness = features[0]['loudness']
        speechiness = features[0]['speechiness']
        tempo = features[0]['tempo']
        time_signature = features[0]['time_signature']

        track = [danceability, acousticness, energy, instrumentalness, liveness, loudness, speechiness, tempo, time_signature]
        return track
    
    return None

In [4]:
# Getting top 50 tracks from user

def fucker(playlist_id):
    sp = login()
    playlist = sp.user_playlist('spotify', playlist_id)
    results = playlist
    
    track_id = []
    track_name = []
    artist = []
    duration = []
    album = []
    popularity = []
    danceability = []
    acousticness = []
    energy = []
    instrumentalness = []
    liveness = []
    loudness = []
    speechiness = []
    tempo = []
    time_signature = []


    for i, items in enumerate(results['tracks']['items']):
            if items and items['track']:
                items2 = getTrackFeatures(items['track']['id'])
                if items2:
                    danceability.append(items2[0])
                    acousticness.append(items2[1])
                    energy.append(items2[2])
                    instrumentalness.append(items2[3])
                    liveness.append(items2[4])
                    loudness.append(items2[5])
                    speechiness.append(items2[6])
                    tempo.append(items2[7])
                    time_signature.append(items2[8])
                
                    track_id.append(items['track']['id'])
                    track_name.append(items['track']['name'])
                    artist.append(items['track']["artists"][0]["name"])
                    duration.append(items['track']["duration_ms"])
                    album.append(items['track']["album"]["name"])
                    popularity.append(items['track']["popularity"])
                
                
    # Create the final df   
    df_favourite = pd.DataFrame({ "track_name": track_name, 
                                 "album": album, 
                                 "track_id": track_id,
                                 "artist": artist, 
                                 "duration": duration, 
                                 "popularity": popularity,
                                 "danceability": danceability,
                                 "acousticness":acousticness,
                                 "energy":energy,
                                 "instrumentalness":instrumentalness,
                                 "liveness":liveness,
                                 "loudness":loudness,
                                 "speechiness":speechiness,
                                 "tempo":tempo,
                                 "time_signature":time_signature})
    
    return df_favourite


In [5]:
# # Getting playlist IDs from each of Spotify's playlists
sp = login()
playlists = sp.user_playlists('spotify')
spotify_playlist_ids = []
while playlists:
    for i, playlist in enumerate(playlists['items']):
        spotify_playlist_ids.append(playlist['uri'][-22:])
    if playlists['next']:
        playlists = sp.next(playlists)
    else:
        playlists = None

In [7]:
finalres = pd.concat(songresults)

In [8]:
finalres.to_csv('FINALsongs.csv',index=False)